In [6]:
!pip install openpyxl SQLAlchemy psycopg2-binary


Defaulting to user installation because normal site-packages is not writeable


In [17]:
!pip install pymongo

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/910.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/910.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/910.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/910.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/910.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/910.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/910.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/910.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/910.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/910.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/910.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/910.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/910.6 kB ? eta -:--:--

In [2]:
# load_relational.py
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError


In [3]:
from sqlalchemy import create_engine, text

DB_URI = "postgresql+psycopg2://postgres@localhost:5432/uci_online_retail"
engine = create_engine(DB_URI)

try:
    with engine.connect() as conn:
        version = conn.execute(text("SELECT version();")).fetchone()
        print("Connected to PostgreSQL:", version[0])
except Exception as e:
    print("Connection failed:", e)


Connected to PostgreSQL: PostgreSQL 16.1, compiled by Visual C++ build 1937, 64-bit


In [4]:
from sqlalchemy import create_engine, inspect

DB_URI = "postgresql+psycopg2://postgres@localhost:5432/uci_online_retail"
engine = create_engine(DB_URI)

inspector = inspect(engine)
print(inspector.get_table_names())


['customers', 'invoices', 'invoiceitems', 'products']


In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError

DB_URI = "postgresql+psycopg2://postgres@localhost:5432/uci_online_retail"
engine = create_engine(DB_URI, pool_size=10, max_overflow=20)

df = pd.read_excel("Online_Retail.xlsx")
df = df.dropna(subset=["InvoiceNo"])
df["Quantity"] = df["Quantity"].astype(float).astype(int)
df["UnitPrice"] = df["UnitPrice"].astype(float)
df["CustomerID"] = df["CustomerID"].fillna("").astype(str)
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], dayfirst=True, errors="coerce")
df = df.head(5000)  # Limit

# Prepare Customers table
customers = (
    df[df["CustomerID"] != ""][["CustomerID", "Country"]]
    .drop_duplicates(subset=["CustomerID"])
    .copy()
)
customers["CustomerID"] = pd.to_numeric(customers["CustomerID"], errors='coerce').dropna().astype(int)
customers.columns = customers.columns.str.lower()

# Prepare Products table 
products = (
    df[["StockCode", "Description", "UnitPrice"]]
    .drop_duplicates(subset=["StockCode"])
    .copy()
)
products.columns = products.columns.str.lower()

# Prepare Invoices table
invoices = df[["InvoiceNo", "InvoiceDate", "CustomerID"]].drop_duplicates()
invoices["CustomerID"] = pd.to_numeric(invoices["CustomerID"], errors='coerce').astype('Int64')
invoices.columns = invoices.columns.str.lower()

# Prepare InvoiceItems table
invoice_items = df[["InvoiceNo", "StockCode", "Quantity", "UnitPrice"]].dropna(subset=["InvoiceNo", "StockCode"])
invoice_items.columns = invoice_items.columns.str.lower()

# Insert into PostgreSQL
with engine.begin() as conn:
    try:
        customers.to_sql("customers", conn, if_exists="append", index=False, method="multi")
        products.to_sql("products", conn, if_exists="append", index=False, method="multi")
        invoices.to_sql("invoices", conn, if_exists="append", index=False, method="multi")
        invoice_items.to_sql("invoiceitems", conn, if_exists="append", index=False, method="multi")
        print(" Inserted records successfully.")
    except SQLAlchemyError as e:
        print("Database error:", e)

# Verify insert
with engine.connect() as conn:
    count = pd.read_sql("SELECT COUNT(*) FROM invoiceitems", conn)
    print("InvoiceItems count:", count.iloc[0, 0])


Database error: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "invoices_pkey"
DETAIL:  Key (invoiceno)=(536591) already exists.

[SQL: INSERT INTO invoices (invoiceno, invoicedate, customerid) VALUES (%(invoiceno_m0)s, %(invoicedate_m0)s, %(customerid_m0)s), (%(invoiceno_m1)s, %(invoicedate_m1)s, %(customerid_m1)s), (%(invoiceno_m2)s, %(invoicedate_m2)s, %(customerid_m2)s), (%(invoiceno_m3)s, %(invoicedate_m3)s, %(customerid_m3)s), (%(invoiceno_m4)s, %(invoicedate_m4)s, %(customerid_m4)s), (%(invoiceno_m5)s, %(invoicedate_m5)s, %(customerid_m5)s), (%(invoiceno_m6)s, %(invoicedate_m6)s, %(customerid_m6)s), (%(invoiceno_m7)s, %(invoicedate_m7)s, %(customerid_m7)s), (%(invoiceno_m8)s, %(invoicedate_m8)s, %(customerid_m8)s), (%(invoiceno_m9)s, %(invoicedate_m9)s, %(customerid_m9)s), (%(invoiceno_m10)s, %(invoicedate_m10)s, %(customerid_m10)s), (%(invoiceno_m11)s, %(invoicedate_m11)s, %(customerid_m11)s), (%(invoiceno_m12)s, %(invoicedate_m12)s, %(customer

## Error due to Multiple Runs and the Items already inserted to sql Database